In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms as trans
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

Загрузка модели:

In [0]:
!pip install pretrainedmodels
from pretrainedmodels import xception 
model = xception()

Инициализация dataloaders:

In [0]:
dataset = ImageFolder('../input/ml3taskfingers', transform =
                      trans.Compose([
                                     trans.ToTensor(),
                                     trans.ToPILImage(), 
                                     trans.RandomRotation(15), 
                                     trans.RandomHorizontalFlip(), 
                                     trans.ColorJitter(0.1, 0.1, 0.1, 0.03), 
                                     trans.ToTensor()
                                    ])
                      )
val_size = 250
dataset,valid = torch.utils.data.random_split(dataset, (len(dataset)-val_size, val_size))
dataloader = DataLoader(dataset, batch_size=8, shuffle=False)
dataloadervalid = DataLoader(valid, batch_size=4, shuffle=False)

Пример данных:

In [0]:
it = iter(dataloader)
t = trans.ToPILImage()

In [0]:
i , a = next(it)
plt.imshow(t(i[0]))

Transfer learning:

In [0]:
for name, child in model.named_children():
    if name=='block5':
        break
    for pars in child.parameters():
        pars.requires_gradu=False
        
model.fc = torch.nn.Linear(2048, 6)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, amsgrad=True)
loss_fn = torch.nn.CrossEntropyLoss()

In [0]:
def acc(ans, pred):
    pred = torch.argmax(pred, dim=1)
    accuracy = torch.mean((ans == pred).float())
    return accuracy.item() 

In [0]:
model = model.cuda()

In [0]:
epoches = 5

In [0]:
for epoch in range(epoches):
    ans, preds = [], []  
    model.train()
    for batch in dataloader:
        X, y = batch
        X = X.cuda()
        y = y.cuda()
        optimizer.zero_grad() 
        output = model(X)
        target = y
        preds.append(output)
        ans.append(target)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        
    ans = torch.cat(ans)
    preds = torch.cat(preds)
    train_accuracy = acc(ans, preds)

    print(f'Epoch {epoch}; train_accuracy {train_accuracy:.4f};')    

In [0]:
model = model.cpu()
model.eval()

In [0]:
ans, preds, f = [], [], []

for batch in dataloadervalid:
    X, y = batch
    X = X
    y = y
    output = model(X)
    target = y
    preds.append(output)
    ans.append(target)
    f.append(X)


In [0]:
ans = torch.cat(ans)
preds = torch.cat(preds)

In [0]:
val_acc = (ans==preds).sum().item() / len(ans)
print(f'val_acc: {val_acc}');

In [0]:
mask = (ans!=preds)
eans, epreds = ans[mask], preds[mask]
feat = torch.cat(f)
ef = feat[mask]

Примеры ошибок:

In [0]:
f, axes = plt.subplots(5,5, figsize=(30, 30))

for x, a, feature, ax in zip(eans, epreds, ef, axes.ravel()):
    ax.imshow(t(feature))
    ax.set_title(f'Predicted as {a}, true is {x}')